In [85]:
import pandas as pd
import numpy as np
df = pd.read_csv('dataset2.csv')

df_new = df.drop(columns=['charges'])
df_new.to_csv('dataset3.csv', index=False)

print(df.head(), df.shape)


   age     sex     bmi  children smoker     region      charges
0   19  female  27.900         0    yes  southwest  16884.92400
1   18    male  33.770         1     no  southeast   1725.55230
2   28    male  33.000         3     no  southeast   4449.46200
3   33    male  22.705         0     no  northwest  21984.47061
4   32    male  28.880         0     no  northwest   3866.85520 (1337, 7)


In [86]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import FunctionTransformer
import math
from sklearn.ensemble import RandomForestRegressor

# separation des features et de la variable cible
X = df.drop('charges', axis=1)
Y = df['charges']
print(f'''verif des dimensions X et Y
      X (dataset sans la variable cible): {X.shape}
      Y (la variable cible) : {Y.shape}''')


# division du dataset en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
print(f''' verif du split 80 20 
80% du dataset : X train -> {X_train.shape}, Y train -> {y_train.shape}
20% du dataset : X test -> {X_test.shape}, Y test -> {y_test.shape}''')

# preprocessing avec standardisation et transformation des categories
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['age', 'bmi', 'children']),
        ('cat', OneHotEncoder(), ['region', 'sex', 'smoker'])
    ]
)


verif des dimensions X et Y
      X (dataset sans la variable cible): (1337, 6)
      Y (la variable cible) : (1337,)
 verif du split 80 20 
80% du dataset : X train -> (1069, 6), Y train -> (1069,)
20% du dataset : X test -> (268, 6), Y test -> (268,)


In [87]:
from sklearn.dummy import DummyRegressor

# création du pipeline dummy
pipeline_dummy = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', DummyRegressor(strategy='mean'))])


# eviter data leakage -> entraîner le pipeline sur les données d'entraînement puis predire
# sur l'ensemble de test avec le meme pipeline
pipeline_dummy.fit(X_train, y_train)
#puis predire y sur l'ensemble de test avec le meme pipeline
y_pred = pipeline_dummy.predict(X_test)
#print(y_pred_dummy)
score = pipeline_dummy.score(X_train, y_train)
print(score)

#test avec les differentes métriques pour le modele dummy : 
#comparaison du y prédit avec le y de test
mse = mean_squared_error(y_test, y_pred)

print(f'Mean Squared Error du modèle dummy: {mse}')

r2 = r2_score(y_test, y_pred)

print(f'Coefficient of Determination R² du modèle dummy: {r2}')

rmse = math.sqrt(mse)
print(f'Root Mean Squared Error (RMSE) du modèle dummy: {rmse}')

0.0
Mean Squared Error du modèle dummy: 185298141.68739662
Coefficient of Determination R² du modèle dummy: -0.008391982350864469
Root Mean Squared Error (RMSE) du modèle dummy: 13612.426003009037


In [88]:
# AVEC LE Modele de regression LASSO


from sklearn.linear_model import Lasso, Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor


# création du pipeline de prétraitement et du modèle Lasso
pipeline_rf = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', RandomForestRegressor())])


#test de différents hyperparam d'alpha pour le lasso
param_grid_lasso = {
    'regressor__n_estimators': [100, 200, 300],
    'regressor__max_depth': [None, 10, 20, 30]
    }

### grid search
grid_lasso = GridSearchCV(pipeline_rf, param_grid_lasso, cv=5)


# eviter data leakage -> entraîner le pipeline sur les données d'entraînement 
grid_lasso.fit(X_train, y_train)
#puis predire y sur l'ensemble de test avec le meme pipeline
y_pred = grid_lasso.predict(X_test)
# print(y_pred)
score = grid_lasso.score(X_train, y_train)
print(score)
#test des les differents metriques sur modele lasso
#comparaison du y prédit avec le y de test
mse = mean_squared_error(y_test, y_pred)

print(f'Mean Squared Error du modèle lasso: {mse}')

r2 = r2_score(y_test, y_pred)

print(f'Coefficient of Determination R² du modèle lasso: {r2}')

rmse = math.sqrt(mse)
print(f'Root Mean Squared Error (RMSE) du modèle lasso: {rmse}')


# Récupérer les meilleurs hyperparamètres
best_params = grid_lasso.best_params_
print("Meilleurs hyperparamètres:", best_params)

# Récupérer le modèle avec les meilleurs hyperparamètres
best_model = grid_lasso.best_estimator_


new_data = pd.DataFrame({
    'age': [35, 40, 30, 60],  # Exemple d'âges
    'sex': ['female', 'male', 'female', 'female'],
    'bmi': [22.5, 30.0, 25.0, 47],
    'children': [1, 2, 0, 5],
    'smoker': ['no', 'yes', 'no', 'yes'],
    'region': ['southwest', 'northeast', 'southeast', 'southeast']
})


# Faire des prédictions avec le modèle optimisé
predictions = best_model.predict(new_data)

# Ajouter les prédictions à votre DataFrame d'origine
new_data['predicted_charges'] = predictions

# Afficher le DataFrame avec les prédictions
print(new_data)

# feature_names = preprocessor.get_feature_names_out()
# print(feature_names)



# 0.6120666150641507
# Mean Squared Error du modèle lasso: 60801052.43566085
# Coefficient of Determination R² du modèle lasso: 0.669120838254013
# Root Mean Squared Error (RMSE) du modèle lasso: 7797.502961567943
# Meilleurs hyperparamètres: {'regressor__max_depth': 20, 'regressor__n_estimators': 200}
#    age     sex   bmi  children smoker     region  predicted_charges
# 0   35  female  22.5         1     no  southwest        8359.378446
# 1   40    male  30.0         2    yes  northeast       30301.875435
# 2   30  female  25.0         0     no  southeast        8670.986965
# 3   60  female  47.0         5    yes  southeast       30971.347171


0.96562116147254
Mean Squared Error du modèle lasso: 21147448.79858042
Coefficient of Determination R² du modèle lasso: 0.8849156412391891
Root Mean Squared Error (RMSE) du modèle lasso: 4598.635536610878
Meilleurs hyperparamètres: {'regressor__max_depth': 10, 'regressor__n_estimators': 200}
   age     sex   bmi  children smoker     region  predicted_charges
0   35  female  22.5         1     no  southwest        9692.151022
1   40    male  30.0         2    yes  northeast       26863.202747
2   30  female  25.0         0     no  southeast        4614.965028
3   60  female  47.0         5    yes  southeast       48629.682448


NameError: name 'b' is not defined

In [ ]:
# Modele de regression RIDGE

# création du pipeline de prétraitement et du modèle dummy
pipeline_ridge = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regression', Ridge())])


#test de différents hyperparam d'alpha pour le lasso
param_grid_ridge = {
    'regression__alpha': [0.01, 0.1, 1, 10, 100]  
}

### grid search
grid_ridge = GridSearchCV(pipeline_ridge, param_grid_ridge, cv=5)


# eviter data leakage -> entraîner le pipeline sur les données d'entraînement 
grid_ridge.fit(X_train, y_train)
#puis predire y sur l'ensemble de test avec le meme pipeline
y_pred = grid_ridge.predict(X_test)
# print(y_pred)
score = grid_ridge.score(X_train, y_train)
print(score)

#test des les differents metriques sur modele lasso
#comparaison du y prédit avec le y de test
mse = mean_squared_error(y_test, y_pred)

print(f'Mean Squared Error du modèle ridge: {mse}')

r2 = r2_score(y_test, y_pred)

print(f'Coefficient of Determination R² du modèle ridge: {r2}')

rmse = math.sqrt(mse)
print(f'Root Mean Squared Error (RMSE) du modèle ridge: {rmse}')


# Récupérer le modèle avec les meilleurs hyperparamètres
best_model = grid_ridge.best_estimator_

print(best_model)


new_data = pd.DataFrame({
    'age': [35, 40, 30, 60],  # Exemple d'âges
    'sex': ['female', 'male', 'female', 'female'],
    'bmi': [22.5, 30.0, 25.0, 47],
    'children': [1, 2, 0, 0],
    'smoker': ['no', 'yes', 'no', 'yes'],
    'region': ['southwest', 'northeast', 'southeast', 'southeast']
})

#  54  female  47.41         0    yes  southeast  63770.42801

# Faire des prédictions avec le modèle optimisé
predictions = best_model.predict(new_data)

# Ajouter les prédictions à votre DataFrame d'origine
new_data['predicted_charges'] = predictions

# Afficher le DataFrame avec les prédictions
print(new_data)